In [ ]:
from bs4 import BeautifulSoup
import mechanize
import pandas as pd
import requests

In [ ]:
dataframe = pd.read_csv('inn.csv')
dataframe['is_valid'] = dataframe.tin.astype(str).str.len() == 10
dataframe

In [ ]:
dataframe['name'] = ''
dataframe['psrn'] = -1
dataframe['psrn_date'] = ''
dataframe['registration_reason_code'] = -1
dataframe['registration_date'] = ''
dataframe['authorized_capital'] = -1

for i, tin in enumerate(dataframe['tin'].values):
    if not dataframe.loc[i, 'is_valid']:
        continue
    br = mechanize.Browser()
    br.open(r'https://www.rusprofile.ru/')
    for form in br.forms():
        try:
            query = form.find_control('query')
        except:
            print(form)
            br.close()
            break
        form.set_value(str(tin), name='query')
        request = form.click(type='submitbutton')
        responce = br.open(request)
        url = responce.geturl()
    else:
        soup = BeautifulSoup(requests.get(url).text, 'html')
        dataframe.loc[i, 'name'] = soup.find('h1').text.strip()
        requisites = [item for item in soup.find('div', {'class': 'company-requisites'}).text.split('\n') if item]
        for j, item in enumerate(requisites):
            if item == 'ОГРН':
                dataframe.loc[i, 'psrn'] = int(requisites[j + 1])
                dataframe.loc[i, 'psrn_date'] = requisites[j + 2]
            if item == 'ИНН/КПП':
                dataframe.loc[i, 'registration_reason_code'] = int(requisites[j + 2])
            if item == 'Дата регистрации':
                dataframe.loc[i, 'registration_date'] = requisites[j + 1]
            if item == 'Уставный капитал':
                _capital = ''.join(requisites[j + 1].split()[:-1])
                _capital = '.'.join(_capital.split(','))
                dataframe.loc[i, 'authorized_capital'] = float(_capital)

dataframe